# Gun Violence Capstone Project

## Data Wrangling 

**1. Import the necessary packages and data**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv('Downloads/Data/gun-violence-data_01-2013_03-2018.csv', parse_dates=['date'])

**2. Preliminary data exploration**

In [3]:
df.shape

(239677, 29)

In [4]:
df.describe()

,incident_id,n_killed,n_injured,congressional_district,latitude,longitude,n_guns_involved,state_house_district,state_senate_district
count,2.396770e+05,239677.000000,239677.000000,227733.000000,231754.000000,231754.000000,140226.000000,200905.000000,207342.00000
mean,5.593343e+05,0.252290,0.494007,8.001265,37.546598,-89.338348,1.372442,55.447132,20.47711
std,2.931287e+05,0.521779,0.729952,8.480835,5.130763,14.359546,4.678202,42.048117,14.20456
min,9.211400e+04,0.000000,0.000000,0.000000,19.111400,-171.429000,1.000000,1.000000,1.00000
25%,3.085450e+05,0.000000,0.000000,2.000000,33.903400,-94.158725,1.000000,21.000000,9.00000
50%,5.435870e+05,0.000000,0.000000,5.000000,38.570600,-86.249600,1.000000,47.000000,19.00000
75%,8.172280e+05,0.000000,1.000000,10.000000,41.437375,-80.048625,1.000000,84.000000,30.00000
max,1.083472e+06,50.000000,53.000000,53.000000,71.336800,97.433100,400.000000,901.000000,94.00000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239677 entries, 0 to 239676
Data columns (total 29 columns):
incident_id                    239677 non-null int64
date                           239677 non-null datetime64[ns]
state                          239677 non-null object
city_or_county                 239677 non-null object
address                        223180 non-null object
n_killed                       239677 non-null int64
n_injured                      239677 non-null int64
incident_url                   239677 non-null object
source_url                     239209 non-null object
incident_url_fields_missing    239677 non-null bool
congressional_district         227733 non-null float64
gun_stolen                     140179 non-null object
gun_type                       140226 non-null object
incident_characteristics       239351 non-null object
latitude                       231754 non-null float64
location_description           42089 non-null object
longitude            

Some of these columns are not very helpful for the scope of this project, such as incident_id, incident_url, notes, and sources. These will need to be dropped.

In [6]:
# Drop irrelevant columns
df.drop(['incident_id','incident_url','source_url','incident_url_fields_missing','location_description','notes','participant_name','participant_relationship', 'sources', 'participant_status', 'address', 'gun_stolen'], axis = 1, inplace = True)

In [7]:
# Observe the missing data
df.isnull().sum()

date                            0
state                           0
city_or_county                  0
n_killed                        0
n_injured                       0
congressional_district      11944
gun_type                    99451
incident_characteristics      326
latitude                     7923
longitude                    7923
n_guns_involved             99451
participant_age             92298
participant_age_group       42119
participant_gender          36362
participant_type            24863
state_house_district        38772
state_senate_district       32335
dtype: int64

There is a lot of missing data but most important columns (n_killed, n_injured, state, city_or_county, date) are all filled. However, incident_characteristics is missing 326 rows so we will need to drop those rows. We can also drop latitude and longitude since we will be using city_or_county for the location.

In [8]:
# Drop missing rows for incident_characteristics
df.dropna(how = 'any', subset = ['incident_characteristics', 'state_house_district', 'congressional_district', 'state_senate_district'], inplace = True)

In [9]:
# Drop latitude and longitude
df.drop(['latitude', 'longitude'], axis = 1, inplace = True)

In [10]:
# Subject-suspect implies that there may not be conclusive proof that the suspect was confirmed to be the shooter.
# There could also have been more shooters in the incident so this column will not be entirely accurate. 
df['participant_type'].value_counts().head(15)

0::Victim                                                                                    49871
0::Victim||1::Subject-Suspect                                                                42728
0::Subject-Suspect                                                                           36280
0::Victim||1::Subject-Suspect||2::Subject-Suspect                                             9211
0::Victim||1::Victim                                                                          7699
0::Subject-Suspect||1::Subject-Suspect                                                        7125
0::Victim||1::Victim||2::Subject-Suspect                                                      5500
0::Victim||1::Subject-Suspect||2::Subject-Suspect||3::Subject-Suspect                         3090
0::Subject-Suspect||1::Subject-Suspect||2::Subject-Suspect                                    2456
0::Victim||1::Victim||2::Subject-Suspect||3::Subject-Suspect                                  1747
0::Victim|

Subject-suspect implies that there may not be conclusive proof that the suspect was confirmed to be the shooter. There could also have been more shooters in the incident so this column will not be entirely accurate. This column should be dropped.

In [11]:
# Drop participant_type since there are too many inconsistencies
df.drop(['participant_type'], axis = 1, inplace = True)

In [12]:
# Number of rows where participant_age_group and gun_type are non-null values.
df[df['participant_age_group'].notnull() & df['gun_type'].notnull()].shape

(92906, 14)

In [13]:
# Number of rows where participant_age and gun_type are non-null values.
df[df['participant_age'].notnull() & df['gun_type'].notnull()].shape

(73098, 14)

Using participant_age_group would provide us with over 20,000 more rows of data compared to participant_age. While the latter is more detailed, it would be difficult to account for outliers where the participants are seniors or toddlers. Therefore, we will be using participant_age_group as one of the feature variables.

In [14]:
# Drop participant_age since we will be using participant_age_group instead
df.drop(['participant_age'], axis = 1, inplace = True)

In [15]:
# View the most common age group values
df.participant_age_group.value_counts().head(15)

0::Adult 18+                                                            79104
0::Adult 18+||1::Adult 18+                                              41051
0::Adult 18+||1::Adult 18+||2::Adult 18+                                11559
0::Teen 12-17                                                            6296
0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+                   4090
1::Adult 18+                                                             3297
0::Adult 18+||1::Teen 12-17                                              1648
0::Teen 12-17||1::Adult 18+                                              1611
0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+     1455
0::Teen 12-17||1::Teen 12-17                                             1409
0:Adult 18+                                                              1080
0::Child 0-11                                                             945
1::Adult 18+||2::Adult 18+                                      

There is a different age group for each participant in the incidents. We will need to parse the data so that the ML algorithm understands how many people there are of each group (Child, Teen, or Adult) in each incident.

In [16]:
# Parse participant_age_group and separate them into 3 new columns.
df['participant_age_group'].fillna('0::Unknown', inplace = True)

def agegroup_cleanup(row):
    agegroup_row_values = []
    
    agegroup_row = str(row).split("||")
    for x in agegroup_row :
        agegroup_row_value = str(x).split("::")
        if len(agegroup_row_value) > 1 :
            agegroup_row_values.append(agegroup_row_value[1])
                
    return agegroup_row_values

agegroupvalues = df.participant_age_group.apply(agegroup_cleanup)
df["agegroup_child"] = agegroupvalues.apply(lambda x: x.count("Child 0-11"))
df["agegroup_teen"] = agegroupvalues.apply(lambda x: x.count("Teen 12-17"))
df["agegroup_adult"] = agegroupvalues.apply(lambda x: x.count("Adult 18+"))

In [17]:
df.incident_characteristics.value_counts().head(10)

Shot - Wounded/Injured                                                                  40099
Shot - Dead (murder, accidental, suicide)                                               19306
Shots Fired - No Injuries                                                               12648
Shot - Wounded/Injured||Drive-by (car to street, car to car)                             4490
Shot - Wounded/Injured||Shot - Dead (murder, accidental, suicide)                        2704
Armed robbery with injury/death and/or evidence of DGU found                             2646
Shot - Wounded/Injured||Armed robbery with injury/death and/or evidence of DGU found     2559
TSA Action                                                                               2378
Non-Shooting Incident||TSA Action                                                        2117
Shots Fired - No Injuries||Drive-by (car to street, car to car)                          2002
Name: incident_characteristics, dtype: int64

There are multiple non-shooting incidents so we will need to slice only the incidents which do not include 'Non-Shooting Incident' in the characteristics.

In [18]:
# Slice the incidents which do not contain 'Non-Shooting Incident'
df = df[df['incident_characteristics'].str.contains('Non-Shooting Incident') == False]

In [19]:
# Create 2 separate columns that contain the first and second characteristics for each row
incident_dict = {}
incident_dict2 = {}
for i, x in df['incident_characteristics'].iteritems():
    x = x.split('|')
    incident_dict[i] = x[0]
    if len(x)>1:
        if x[1] == '':
            incident_dict2[i] = x[2]
        else:
            incident_dict2[i] = x[1]
df2 = pd.DataFrame([incident_dict, incident_dict2])
df2 = df2.transpose()
df2 = pd.concat([df, df2], axis = 'columns')
df2.rename(columns={0:'primary_characteristic', 1: 'secondary_characteristic'}, inplace=True)

In [20]:
# Parse the incident characteristics and store the 20 most common descriptions for graphical purposes
big_text = "||".join(df['incident_characteristics']).split("||")
IC_series = pd.Series(big_text)
IC_series.value_counts().head(20)

Shot - Wounded/Injured                                            79319
Shot - Dead (murder, accidental, suicide)                         44132
Shots Fired - No Injuries                                         28778
Armed robbery with injury/death and/or evidence of DGU found      14997
Officer Involved Incident                                         11943
Drive-by (car to street, car to car)                              11556
Possession (gun(s) found during commission of other crimes)       10355
Domestic Violence                                                  7615
Institution/Group/Business                                         7017
Accidental Shooting                                                6744
Brandishing/flourishing/open carry/lost/found                      6668
Home Invasion                                                      5684
Possession of gun by felon or prohibited person                    5507
Defensive Use                                                   

Let's create separate columns for the dates and a total 'victims' column for injuries and killed.

In [21]:
# Convert 'date' column into datetime and create additional columns
df['date'] = pd.to_datetime(df.date)
df['year'] = df.date.dt.year
df['month'] = df.date.dt.month
df['monthday'] = df.date.dt.day
df['weekday'] = df.date.dt.weekday

Now let's take a look into the different gun types used in this dataset.

In [22]:
df['gun_type'].value_counts(dropna=False).head(15)

NaN                       81657
0::Unknown                65702
0::Handgun                 5142
0::9mm                     1393
0::Unknown||1::Unknown     1255
0::Shotgun                 1055
0::22 LR                    790
0::Rifle                    740
0::40 SW                    585
0::45 Auto                  507
0::Handgun||1::Handgun      495
0::380 Auto                 455
0::38 Spl                   344
0:Unknown                   215
0::223 Rem [AR-15]          207
Name: gun_type, dtype: int64

In [23]:
(93020 + 81043 + 1526)/len(df['gun_type'])

1.072587443343555

At least 90.2% of gun_type entries are either classified as 'Unknown' or are missing. However, since the type of gun may have valuable correlation to the response variables (n_killed, n_injured), it may be better to keep them.

In [24]:
# Source: https://www.kaggle.com/shivamb/stop-gun-violence-updated-exploration
# Create a new dictionary that presents the number of times each weapon is used and how many killed/injured for each one.
df['gun_type'].fillna('0::Unknown', inplace = True)
gt = df.groupby(by=['gun_type']).agg({'n_killed': 'sum', 'n_injured' : 'sum', 'state' : 'count'}).reset_index().rename(columns={'state':'count'})
results = {}
for i, each in gt.iterrows():
    wrds = each['gun_type'].split("||")
    for wrd in wrds:
        if "Unknown" in wrd:
            continue
        wrd = wrd.replace("::",":").replace("|1","")
        gtype = wrd.split(":")[1]
        if gtype not in results: 
            results[gtype] = {'killed' : 0, 'injured' : 0, 'used' : 0}
        results[gtype]['killed'] += each['n_killed']
        results[gtype]['injured'] +=  each['n_injured']
        results[gtype]['used'] +=  each['count']

gun_names = list(results.keys())
used = [each['used'] for each in list(results.values())]
killed = [each['killed'] for each in list(results.values())]
injured = [each['injured'] for each in list(results.values())]

In [25]:
df['city_or_county'].value_counts().head(20)

Chicago         9970
Baltimore       2665
New Orleans     2345
Milwaukee       2269
Saint Louis     2223
Jacksonville    2210
Memphis         2045
Columbus        1899
Houston         1796
Indianapolis    1575
Cleveland       1498
Detroit         1466
Oakland         1397
Kansas City     1193
Las Vegas       1176
Jackson         1159
San Antonio     1138
Brooklyn        1127
Charlotte       1108
Richmond        1100
Name: city_or_county, dtype: int64

In [26]:
# Create a list of the top 15 most violent cities in the data
city_list = df['city_or_county'].value_counts().head(15).index.tolist()

# Keep only rows that contain one of the top 15 cities
df = df[df['city_or_county'].apply(lambda x: x in city_list)]

In [27]:
df['n_guns_involved'].value_counts(dropna = False)

 1.0     17696
NaN      17357
 2.0       549
 3.0        88
 4.0        18
 5.0        11
 6.0         2
 15.0        1
 43.0        1
 12.0        1
 20.0        1
 11.0        1
 8.0         1
Name: n_guns_involved, dtype: int64

In [28]:
(21014 + 19215)/ len(df['n_guns_involved'])

1.126011140034148

Since this dataframe only includes shooting incidents, it would be safe to assume that the missing values are cases where at least 1 gun is used. However, since over 98% of the rows would be categorized as either '1' or missing, it would be best to simply not use the column.

In [29]:
df.drop('n_guns_involved', axis = 1, inplace = True)

Next, let's take a look at the gender column which has quite a few missing values.

In [30]:
df['participant_gender'].value_counts(dropna = False).head()

0::Male                      15226
0::Male||1::Male              7288
NaN                           4254
0::Male||1::Male||2::Male     2002
0::Female                     1476
Name: participant_gender, dtype: int64

In [31]:
5001/len(df['participant_gender'])

0.13997816777227307

12% of the data is missing but it could be valuable information. More importantly, it would be best to create dummy variable columns for each gender.

In [32]:
df['participant_gender'].fillna('0::Unknown', inplace = True)

def gender_cleanup(row):
    gender_row_values = []
    
    gender_row = str(row).split("||")
    for x in gender_row :
        gender_row_value = str(x).split("::")
        if len(gender_row_value) > 1 :
            gender_row_values.append(gender_row_value[1])
                
    return gender_row_values

participant_genders = df.participant_gender.apply(gender_cleanup)
df["participant_gender_total"] = participant_genders.apply(lambda x: len(x))
df["participant_gender_male"] = participant_genders.apply(lambda x: x.count("Male"))
df["participant_gender_female"] = participant_genders.apply(lambda x: x.count("Female"))
df["participant_gender_unknown"] = participant_genders.apply(lambda x: x.count("Unknown"))

Now let's take a look at congressional_district, state_house_district, and state_senate_district for their missing values.

In [33]:
df['congressional_district'].value_counts(dropna=False).head(10)

7.0     7931
1.0     5551
4.0     4594
2.0     4410
9.0     2828
5.0     2724
13.0    2201
3.0     2198
11.0    1302
18.0     779
Name: congressional_district, dtype: int64

In [34]:
2225/len(df['congressional_district'])

0.06227782909284295

Since only 5.4% of the data is missing, we can simply drop the missing data..

In [35]:
df['state_house_district'].value_counts(dropna=False).head(10)

10.0    2200
18.0    1882
9.0     1277
13.0    1163
8.0     1124
40.0    1082
6.0      978
14.0     957
93.0     928
16.0     926
Name: state_house_district, dtype: int64

In [36]:
7627/len(df['state_house_district'])

0.21348000111960141

18.5% of the data is missing so it would be best to simply drop the missing data.

In [37]:
df['state_senate_district'].value_counts(dropna=False).head(10)

5.0     4423
4.0     3858
9.0     3674
3.0     2727
15.0    1971
6.0     1945
13.0    1683
21.0    1172
33.0    1049
1.0      993
Name: state_senate_district, dtype: int64

In [38]:
6638/len(df['state_senate_district'])

0.18579785596327708